<a href="https://colab.research.google.com/github/perlati/lab-neural-networks/blob/master/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

tictac = pd.read_csv('tic-tac-toe.csv')  # Load the CSV file into a DataFrame

# Step 3: Inspect the dataset
print("Shape of dataset:", tictac.shape)
print("\nFirst 5 rows:")
print(tictac.head())
print("\nColumn information:")
print(tictac.info())
print("\nUnique values in each column:")
for col in tictac.columns:
    print(f"{col}: {tictac[col].unique()}")


# Step 4: Convert categorical values to numeric
label_encoders = {}
for col in tictac.columns:
    le = LabelEncoder()
    tictac[col] = le.fit_transform(tictac[col])
    label_encoders[col] = le  # store encoders for potential inverse_transform later

# Step 5: Separate inputs (X) and output (y)
X = tictac.drop('class', axis=1)  # assuming 'class' is the target column
y = tictac['class']

# Step 6: Normalize the input data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Step 7: Convert back to DataFrame (optional, for readability)
X_normalized = pd.DataFrame(X_normalized, columns=X.columns)

# Step 8: Check results
print("\nNormalized input sample:")
print(X_normalized.head())
print("\nTarget sample:")
print(y.head())



Shape of dataset: (958, 10)

First 5 rows:
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True

Column information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None

Unique values in each column:
TL: ['x' 'o' 'b']
TM: ['x' 'o' 'b']
TR: ['x' 'o' 'b']
ML: ['x' 'o' 'b']
MM: ['o' 'b' 'x']
MR: ['o

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [16]:
# --- Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os, random


# --- Reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# --- Load & quick prep (from Step 1 but done properly to avoid leakage)
df = pd.read_csv("tic-tac-toe.csv")

# Encode every column (works for features and 'class')
encoders = {}
for col in df.columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

# Separate inputs and output
X = df.drop("class", axis=1).values
y = df["class"].values  # integers for sparse_categorical_crossentropy

# Train/test split (stratify to preserve class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed, stratify=y
)

# Normalize using stats from TRAIN ONLY (avoid data leakage)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# --- Build the model
num_features = X_train.shape[1]
num_classes  = len(np.unique(y_train))

model = keras.Sequential([
    layers.Input(shape=(num_features,)),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(num_classes, activation="softmax")  # Softmax for class probabilities
])

# --- Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  # integer labels
    metrics=["accuracy"]
)

# --- Train
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=16,
    verbose=1
)

# --- Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")

# --- Save model
# Will create a TensorFlow SavedModel directory named 'tic-tac-toe.model'
model.save("tic-tac-toe.keras")
print("Model saved to 'tic-tac-toe.model'")


Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6597 - loss: 0.6405 - val_accuracy: 0.6364 - val_loss: 0.6505
Epoch 2/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6653 - loss: 0.6235 - val_accuracy: 0.6364 - val_loss: 0.6448
Epoch 3/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6653 - loss: 0.6115 - val_accuracy: 0.6364 - val_loss: 0.6394
Epoch 4/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6653 - loss: 0.6005 - val_accuracy: 0.6364 - val_loss: 0.6335
Epoch 5/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6667 - loss: 0.5907 - val_accuracy: 0.6429 - val_loss: 0.6277
Epoch 6/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6774 - loss: 0.5814 - val_accuracy: 0.6429 - val_loss: 0.6227
Epoch 7/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6935 - loss: 0.5722 - val_accuracy: 0.6558 - val_loss: 0.6181
Epoch 8/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7068 - loss: 0.5633 - val_accuracy: 0.6623 - val_loss

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [18]:
# --- Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf

# --- Rebuild the exact preprocessing used for training
seed = 42
df = pd.read_csv("tic-tac-toe.csv")

encoders = {}
for col in df.columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

X = df.drop("class", axis=1).values
y = df["class"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed, stratify=y
)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# --- Load saved model and predict a few random rows from the test set
model = tf.keras.models.load_model("tic-tac-toe.keras")

np.random.seed(seed)
idx = np.random.choice(len(X_test), size=5, replace=False)

probs = model.predict(X_test[idx], verbose=0)
preds = probs.argmax(axis=1)

for i, j in enumerate(idx):
    print(f"Row #{j:>3}  |  Pred: {preds[i]}  (p={probs[i][preds[i]]:.3f})  |  True: {y_test[j]}")


Row # 45  |  Pred: 1  (p=0.668)  |  True: 1
Row #136  |  Pred: 1  (p=0.765)  |  True: 1
Row # 76  |  Pred: 1  (p=0.945)  |  True: 1
Row #143  |  Pred: 0  (p=0.808)  |  True: 1
Row #113  |  Pred: 1  (p=0.591)  |  True: 1


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Reuse X_train, X_test, y_train, y_test, scaler from above

def build_model(n_units=(32,16), lr=1e-3, input_dim=X_train.shape[1], n_classes=len(np.unique(y_train))):
    model = keras.Sequential([layers.Input(shape=(input_dim,))])
    for u in n_units:
        model.add(layers.Dense(u, activation="relu"))
    model.add(layers.Dense(n_classes, activation="softmax"))
    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

configs = [
    {"n_units": (32,16),      "lr": 1e-3, "epochs": 50},
    {"n_units": (64,32,16),   "lr": 1e-3, "epochs": 50},
    {"n_units": (64,32,16),   "lr": 3e-4, "epochs": 75},
    {"n_units": (128,64,32),  "lr": 3e-4, "epochs": 100},
    {"n_units": (32,),        "lr": 1e-3, "epochs": 75},
]

results = []

for cfg in configs:
    model = build_model(cfg["n_units"], cfg["lr"])
    es = keras.callbacks.EarlyStopping(
        monitor="val_accuracy", patience=10, restore_best_weights=True
    )
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=cfg["epochs"],
        batch_size=16,
        verbose=0,
        callbacks=[es]
    )
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    best_val_acc = max(history.history["val_accuracy"])
    results.append({
        "n_units": cfg["n_units"],
        "lr": cfg["lr"],
        "epochs_ran": len(history.history["accuracy"]),
        "best_val_acc": float(best_val_acc),
        "test_acc": float(test_acc),
        "test_loss": float(test_loss),
        "model": model,
    })
    print(f"Done {cfg} -> best val_acc={best_val_acc:.4f}, test_acc={test_acc:.4f}")

# Sort and show the top runs by test accuracy
results_sorted = sorted(results, key=lambda r: r["test_acc"], reverse=True)
top = results_sorted[0]
print("\nBest configuration:")
print({
    "n_units": top["n_units"],
    "lr": top["lr"],
    "epochs_ran": top["epochs_ran"],
    "best_val_acc": top["best_val_acc"],
    "test_acc": top["test_acc"],
})

# Save the best model (overwrites the original if you want)
top["model"].save("tic-tac-toe.keras")
print("Best model saved to 'tic-tac-toe.keras'")


Done {'n_units': (32, 16), 'lr': 0.001, 'epochs': 50} -> best val_acc=0.7857, test_acc=0.7760
Done {'n_units': (64, 32, 16), 'lr': 0.001, 'epochs': 50} -> best val_acc=0.8312, test_acc=0.8177
Done {'n_units': (64, 32, 16), 'lr': 0.0003, 'epochs': 75} -> best val_acc=0.7792, test_acc=0.7760
Done {'n_units': (128, 64, 32), 'lr': 0.0003, 'epochs': 100} -> best val_acc=0.7922, test_acc=0.7812
Done {'n_units': (32,), 'lr': 0.001, 'epochs': 75} -> best val_acc=0.6753, test_acc=0.6771

Best configuration:
{'n_units': (64, 32, 16), 'lr': 0.001, 'epochs_ran': 50, 'best_val_acc': 0.8311688303947449, 'test_acc': 0.8177083134651184}
Best model saved to 'tic-tac-toe.keras'


**Which approach(es) did you find helpful to improve your model performance?**

In [24]:
print("Increasing the number of hidden layers from two to three (64→32→16 neurons) while keeping the Adam learning rate at 0.001 improved both validation and test accuracy from ~77% to ~82%. Lowering the learning rate or adding more layers beyond this didn’t yield further gains. Early stopping also helped stabilize validation accuracy.")

Increasing the number of hidden layers from two to three (64→32→16 neurons) while keeping the Adam learning rate at 0.001 improved both validation and test accuracy from ~77% to ~82%. Lowering the learning rate or adding more layers beyond this didn’t yield further gains. Early stopping also helped stabilize validation accuracy.
